# Initial definitions

In [3]:
%env HADOOP_VERSION     2.10.1
%env HADOOP_PATH hadoop-2.10.1

env: HADOOP_VERSION=2.10.1
env: HADOOP_PATH=hadoop-2.10.1


# Preparing the environment

## Downloading Hadoop

In [ ]:
!wget http://ftp.unicamp.br/pub/apache/hadoop/common/hadoop-${HADOOP_VERSION}/hadoop-${HADOOP_VERSION}.tar.gz -q --show-progress

## Extracting compressed files and removing .tar

In [ ]:
# !rm ${HADOOP_PATH} -r
!tar -xvf hadoop-${HADOOP_VERSION}.tar.gz >/dev/null 
!rm       hadoop-${HADOOP_VERSION}.tar.gz

## Discovering the Java path

In [ ]:
!dirname $(dirname $(readlink -f $(which javac)))

## Setting the Java path envvar

We also added it to user's .bashrc so it will be loaded as the nodes perform ssh connections.

In [1]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64

env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64


In [4]:
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " >> ~/.bashrc
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " >> ~/.profile
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " >> ${HADOOP_PATH}/etc/hadoop/hadoop-env.sh

# Hadoop in Standalone Mode (local)

## MapReduce in the local filesystem - word count example

In [ ]:
!${HADOOP_PATH}/bin/hadoop jar ${HADOOP_PATH}/share/hadoop/mapreduce/hadoop-mapreduce-examples-${HADOOP_VERSION}.jar wordcount \
                               ./resources/examples/newyorknewyork.txt ./output

### Listing files in the output folder

In [ ]:
!ls ./output/

### Reading output file

In [ ]:
! cat ./output/part-r-00000

# Hadoop in Pseudo-Distributed Mode

## Preparing the environment

### Starting sshd server

Check `/binder/postBuild` and `/resources/configs/ssh/sshd_config` files for more details

In [9]:
!/usr/sbin/sshd -f resources/configs/ssh/sshd_config 

### Adding names to know hosts 

Commands below stablish ssh connections to used host names/ips. This step avoids yes/no host confirmation.

In [10]:
!ssh -o "StrictHostKeyChecking no" $USER@localhost -p 8822 -C "exit" 
!ssh -o "StrictHostKeyChecking no" $USER@0.0.0.0   -p 8822 -C "exit"

### Adding ssh options to Hadoop via envvar

* connecting in a diferent port (`-p 8822`)
* avoiding host key checking (`-o StrictHostKeyChecking=no`)

In [11]:
%env HADOOP_SSH_OPTS= -o StrictHostKeyChecking=no -p 8822

env: HADOOP_SSH_OPTS=-o StrictHostKeyChecking=no -p 8822


In [12]:
%env PDSH_RCMD_TYPE ssh

env: PDSH_RCMD_TYPE=ssh


### Copying configurations files to Hadoop folder

Check the configuration files accordingly to the Hadoop version. 
Refer to the `/resources/configs/hadoop/<version>`.

In [5]:
!cp resources/configs/hadoop/${HADOOP_VERSION}/core-site.xml   ${HADOOP_PATH}/etc/hadoop/
!cp resources/configs/hadoop/${HADOOP_VERSION}/hdfs-site.xml   ${HADOOP_PATH}/etc/hadoop/

## Formatting the filesystem

In [6]:
!${HADOOP_PATH}/bin/hdfs namenode -format -force -nonInteractive

22/09/25 17:07:14 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = jupyter-lsteffenel-2dbinderhub-2dhadoop-2dwf5rlram/10.0.4.196
STARTUP_MSG:   args = [-format, -force, -nonInteractive]
STARTUP_MSG:   version = 2.10.1
STARTUP_MSG:   classpath = /home/jovyan/hadoop-2.10.1/etc/hadoop:/home/jovyan/hadoop-2.10.1/share/hadoop/common/lib/commons-math3-3.1.1.jar:/home/jovyan/hadoop-2.10.1/share/hadoop/common/lib/apacheds-i18n-2.0.0-M15.jar:/home/jovyan/hadoop-2.10.1/share/hadoop/common/lib/commons-compress-1.19.jar:/home/jovyan/hadoop-2.10.1/share/hadoop/common/lib/snappy-java-1.0.5.jar:/home/jovyan/hadoop-2.10.1/share/hadoop/common/lib/protobuf-java-2.5.0.jar:/home/jovyan/hadoop-2.10.1/share/hadoop/common/lib/commons-lang-2.6.jar:/home/jovyan/hadoop-2.10.1/share/hadoop/common/lib/api-util-1.0.0-M20.jar:/home/jovyan/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar:/home/jovyan

## Starting DFS (NameNode, SecondaryNameNode, and DataNode daemons)

In [13]:
!${HADOOP_PATH}/sbin/start-dfs.sh
!jps

Starting namenodes on [localhost]
localhost: starting namenode, logging to /home/jovyan/hadoop-2.10.1/logs/hadoop-jovyan-namenode-jupyter-lsteffenel-2dbinderhub-2dhadoop-2dwf5rlram.out
localhost: starting datanode, logging to /home/jovyan/hadoop-2.10.1/logs/hadoop-jovyan-datanode-jupyter-lsteffenel-2dbinderhub-2dhadoop-2dwf5rlram.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /home/jovyan/hadoop-2.10.1/logs/hadoop-jovyan-secondarynamenode-jupyter-lsteffenel-2dbinderhub-2dhadoop-2dwf5rlram.out
2353 SecondaryNameNode
2180 DataNode
2089 NameNode
2476 Jps


## MapReduce - Word count example 

### Creating folders in the distributed file system

In [14]:
!${HADOOP_PATH}/bin/hdfs dfs -mkdir /user/
!${HADOOP_PATH}/bin/hdfs dfs -mkdir /user/rt0902/
!${HADOOP_PATH}/bin/hdfs dfs -mkdir /user/rt0902/input/

### Copying a file to a folder in the distributed file system

In [ ]:
!${HADOOP_PATH}/bin/hdfs dfs -put ./resources/examples/newyorknewyork.txt /user/rt0902/input/

### Listing files in a folder of the distributed file system

In [ ]:
!${HADOOP_PATH}/bin/hdfs dfs -ls /user/rt0902/input/

### Retrieving the contents of a file in the distributed file system

In [ ]:
!${HADOOP_PATH}/bin/hdfs dfs -cat /user/rt0902/input/newyorknewyork.txt

### Running MapReduce job in Pseudo-Distributed Mode

In [ ]:
!./${HADOOP_PATH}/bin/hadoop jar  ./${HADOOP_PATH}/share/hadoop/mapreduce/hadoop-mapreduce-examples-${HADOOP_VERSION}.jar wordcount \
                                /user/rt0902/input /user/rt0902/output

### Listing files in the output folder

In [ ]:
!./${HADOOP_PATH}/bin/hdfs dfs -ls /user/rt0902/output/

### Reading output file

In [ ]:
!./${HADOOP_PATH}/bin/hdfs dfs -cat /user/rt0902/output/part-r-00000

# Starting YARN in Pseudo-Distributed Mode

## Preparing the environment

### Copying configurations files to Hadoop folder

In [ ]:
!cp resources/configs/hadoop/${HADOOP_VERSION}/mapred-site.xml ${HADOOP_PATH}/etc/hadoop/
!cp resources/configs/hadoop/${HADOOP_VERSION}/yarn-site.xml   ${HADOOP_PATH}/etc/hadoop/

## Starting YARN

In [ ]:
!${HADOOP_PATH}/sbin/start-yarn.sh
!jps

## MapReduce via YARN - Word count example 

In [ ]:
!./${HADOOP_PATH}/bin/yarn jar  ./${HADOOP_PATH}/share/hadoop/mapreduce/hadoop-mapreduce-examples-${HADOOP_VERSION}.jar wordcount \
                                /user/rt0902/input /user/rt0902/output2

### Listing files in the output folder

In [ ]:
!./${HADOOP_PATH}/bin/hdfs dfs -ls /user/rt0902/output2/

### Reading output file

In [ ]:
!./${HADOOP_PATH}/bin/hdfs dfs -cat /user/rt0902/output2/part-r-00000